In [1]:
from nustar_pysolar import planning, io
import astropy.units as u
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2020_252.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [13]:
tstart = '2020-09-12T08:30:00'
tend = '2020-09-13T01:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [11]:
pa = planning.get_nustar_roll(tstart, 0)
print(tstart)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

2020-09-12T08:30:00
NuSTAR Roll angle for Det0 in NE quadrant: 23.630807678782805 deg


In [20]:
orbits

[[<Time object: scale='utc' format='isot' value=2020-09-12T09:04:40.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T10:06:00.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T10:41:20.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T11:42:40.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T12:18:00.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T13:19:20.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T13:54:40.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T14:55:50.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T15:31:20.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T16:32:30.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T17:08:00.000>,
  <Time object: scale='utc' format='isot' value=2020-09-12T18:09:10.000>],
 [<Time object: scale='utc' format='isot' value=2020-09-12T18:44:40.000>,
  <Time object: scale='utc' form

In [31]:
# Get the solar parameter
from sunpy.coordinates import sun

angular_size = sun.angular_radius(t='now')
dx = angular_size.arcsec
print(dx)

952.5099666382587


In [35]:
offset = [dx, 0]*u.arcsec
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
    print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.3f} Dec (deg): {sky_pos[1]:8.3f}')
    print("")

Orbit: 0
Orbit start: 2020-09-12 09:04:40.000 Orbit end: 2020-09-12 10:06:00.000
Aim time: 2020-09-12 09:35:20.000 RA (deg):  170.384 deg Dec (deg):    4.145 deg

Orbit: 1
Orbit start: 2020-09-12 10:41:20.000 Orbit end: 2020-09-12 11:42:40.000
Aim time: 2020-09-12 11:12:00.000 RA (deg):  170.444 deg Dec (deg):    4.119 deg

Orbit: 2
Orbit start: 2020-09-12 12:18:00.000 Orbit end: 2020-09-12 13:19:20.000
Aim time: 2020-09-12 12:48:40.000 RA (deg):  170.504 deg Dec (deg):    4.094 deg

Orbit: 3
Orbit start: 2020-09-12 13:54:40.000 Orbit end: 2020-09-12 14:55:50.000
Aim time: 2020-09-12 14:25:15.000 RA (deg):  170.565 deg Dec (deg):    4.068 deg

Orbit: 4
Orbit start: 2020-09-12 15:31:20.000 Orbit end: 2020-09-12 16:32:30.000
Aim time: 2020-09-12 16:01:55.000 RA (deg):  170.625 deg Dec (deg):    4.043 deg

Orbit: 5
Orbit start: 2020-09-12 17:08:00.000 Orbit end: 2020-09-12 18:09:10.000
Aim time: 2020-09-12 17:38:35.000 RA (deg):  170.685 deg Dec (deg):    4.017 deg

Orbit: 6
Orbit start: 

# This is where you actually make the Mosaic for Orbit 1

In [ ]:
# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.
orbit = orbits[0].copy()
print(orbit)
#...adjust the index above to get the correct orbit. Then uncomment below.

planning.make_mosaic(orbit, make_regions=True, outfile='orbit1_mosaic.txt', write_output=True)

